In [ ]:
from data_loader import *

: 

In [ ]:
#cleaning fct_order_items.csv
data_loader = DataLoader("D:\Deloitte\DIH-X-AUC-Hackathon\data\Inventory Management")
df = data_loader.load_csv("fct_order_items.csv")

D:\Deloitte\DIH-X-AUC-Hackathon\src\models\data_loader.py:56: DtypeWarning: Columns (9,11,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, parse_dates=parse_dates)


Successfully loaded fct_order_items.csv: 1999341 rows


In [ ]:
#knowing the shape, the columns, and the info of the dataset
print(df.shape)
print(df.columns.tolist())
df.head()

(1999341, 23)
['id', 'user_id', 'created', 'updated', 'title', 'campaign_id', 'commission_amount', 'cost', 'discount_amount', 'external_id', 'group', 'instructions', 'item_id', 'order_id', 'points_earned', 'points_redeemed', 'price', 'quantity', 'redemptions', 'removed_ingredients', 'add_on_ids', 'status', 'vat_amount']


,id,user_id,created,updated,title,campaign_id,commission_amount,cost,discount_amount,external_id,group,instructions,item_id,order_id,points_earned,points_redeemed,price,quantity,redemptions,removed_ingredients,add_on_ids,status,vat_amount
0,60824,512,1613139429,1708921636,Latte,NaN,1.50,50.0,0.0,NaN,NaN,NaN,60822,60825.0,NaN,NaN,25.0,2,0,NaN,NaN,Pending,NaN
1,60837,514,1613144783,1708921895,The Classic Meal,NaN,5.25,175.0,0.0,NaN,NaN,NaN,59999,60841.0,NaN,NaN,175.0,1,0,NaN,59995,Pending,NaN
2,60838,514,1613144783,1708921895,The Classic Meal,NaN,4.80,160.0,0.0,NaN,NaN,NaN,59999,60841.0,NaN,NaN,160.0,1,0,NaN,NaN,Pending,NaN
3,60839,514,1613144783,1708921895,The Classic Meal,NaN,4.80,160.0,0.0,NaN,NaN,NaN,59999,60841.0,NaN,NaN,160.0,1,0,NaN,NaN,Pending,NaN
4,60840,514,1613144783,1708921895,The Classic Meal,NaN,4.80,160.0,0.0,NaN,NaN,NaN,59999,60841.0,NaN,NaN,160.0,1,0,NaN,NaN,Pending,NaN


In [ ]:
#showing important info of the dataset
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1999341 entries, 0 to 1999340
Data columns (total 23 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   id                   1999341 non-null  int64  
 1   user_id              1999341 non-null  int64  
 2   created              1999341 non-null  int64  
 3   updated              1999341 non-null  int64  
 4   title                1999341 non-null  object 
 5   campaign_id          1941 non-null     float64
 6   commission_amount    1995417 non-null  float64
 7   cost                 1999341 non-null  float64
 8   discount_amount      1999341 non-null  float64
 9   external_id          4111 non-null     object 
 10  group                710286 non-null   float64
 11  instructions         2362 non-null     object 
 12  item_id              1999341 non-null  int64  
 13  order_id             1999179 non-null  float64
 14  points_earned        1990916 non-null  float64
 15

In [ ]:
#dealing with missing data
#convert types
#any missing id will be filled with zero or empty. some columns that their null means no information given like price will be removed
df['campaign_id'] = df['campaign_id'].fillna('none')
df['external_id'] = df['external_id'].fillna('none')
df['removed_ingredients'] = df['removed_ingredients'].fillna('none')
df['add_on_ids'] = df['add_on_ids'].fillna('0')
df['order_id'] = df['order_id'].fillna(0)
df['commission_amount'] = df['commission_amount'].fillna(0)
df = df.dropna(subset=['price', 'status'])
df['group'] = df['group'].fillna(-1)
df['group'] = df['group'] + 1
df['instructions'] = df['instructions'].fillna('none')
df['points_earned'] = df['points_earned'].fillna(0)
df['points_redeemed'] = df['points_redeemed'].fillna(0)
df['vat_amount'] = df['vat_amount'].fillna(0)

In [ ]:
#checking duplicates
df.drop_duplicates(subset=['id'],inplace=True)

In [ ]:
#removing outliers
cols_to_check = [col for col in df.select_dtypes(include=[np.number]).columns 
                 if not any(x in col.lower() for x in ['id', 'created', 'updated'])]
z_scores = np.abs(stats.zscore(df[cols_to_check]))    
df = df[(z_scores < 2).all(axis=1)]

In [ ]:
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 1974592 entries, 0 to 1999339
Data columns (total 23 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   id                   1974592 non-null  int64  
 1   user_id              1974592 non-null  int64  
 2   created              1974592 non-null  int64  
 3   updated              1974592 non-null  int64  
 4   title                1974592 non-null  object 
 5   campaign_id          1974592 non-null  object 
 6   commission_amount    1974592 non-null  float64
 7   cost                 1974592 non-null  float64
 8   discount_amount      1974592 non-null  float64
 9   external_id          1974592 non-null  object 
 10  group                1974592 non-null  float64
 11  instructions         1974592 non-null  object 
 12  item_id              1974592 non-null  int64  
 13  order_id             1974592 non-null  float64
 14  points_earned        1974592 non-null  float64
 15  poi

In [ ]:
#converting types (unix timestamps to datetime)
df['created'] = pd.to_datetime(df['created'],unit='s').dt.date
df['updated'] = pd.to_datetime(df['updated'],unit='s').dt.date

In [ ]:
df.head()

,id,user_id,created,updated,title,campaign_id,commission_amount,cost,discount_amount,external_id,group,instructions,item_id,order_id,points_earned,points_redeemed,price,quantity,redemptions,removed_ingredients,add_on_ids,status,vat_amount
0,60824,512,2021-02-12,2024-02-26,Latte,none,1.50,50.0,0.0,none,0.0,none,60822,60825.0,0.0,0.0,25.0,2,0,none,0,Pending,0.0
1,60837,514,2021-02-12,2024-02-26,The Classic Meal,none,5.25,175.0,0.0,none,0.0,none,59999,60841.0,0.0,0.0,175.0,1,0,none,59995,Pending,0.0
2,60838,514,2021-02-12,2024-02-26,The Classic Meal,none,4.80,160.0,0.0,none,0.0,none,59999,60841.0,0.0,0.0,160.0,1,0,none,0,Pending,0.0
3,60839,514,2021-02-12,2024-02-26,The Classic Meal,none,4.80,160.0,0.0,none,0.0,none,59999,60841.0,0.0,0.0,160.0,1,0,none,0,Pending,0.0
4,60840,514,2021-02-12,2024-02-26,The Classic Meal,none,4.80,160.0,0.0,none,0.0,none,59999,60841.0,0.0,0.0,160.0,1,0,none,0,Pending,0.0


In [ ]:
df.to_csv('D:/Deloitte/DIH-X-AUC-Hackathon/data/Cleaned Data/fct_order_items.csv')

In [ ]:
#cleaning fct_bonus_codes.csv
data_loader = DataLoader("D:\Deloitte\DIH-X-AUC-Hackathon\data\Inventory Management")
df = data_loader.load_csv("fct_bonus_codes.csv")

Successfully loaded fct_bonus_codes.csv: 6 rows


In [ ]:
#knowing the shape, the columns, and the info of the dataset
print(df.shape)
print(df.columns.tolist())
df.head()

(6, 9)
['id', 'user_id', 'created', 'updated', 'end_date_time', 'start_date_time', 'place_id', 'points', 'redemptions']


,id,user_id,created,updated,end_date_time,start_date_time,place_id,points,redemptions
0,61628,423,1614960382,1615136541,1615248000,1614902400,NaN,150,NaN
1,61922,423,1615559087,1637324088,1640822400,1615420800,59897.0,200,NaN
2,61940,423,1615564762,1622661882,1640908800,1615420800,NaN,300,NaN
3,63306,423,1617184682,1619715679,1640908800,1617148800,NaN,200,NaN
4,63379,417,1617265478,1619514716,1640908800,1617235200,NaN,200,NaN


In [ ]:
#showing important info of the dataset
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               6 non-null      int64  
 1   user_id          6 non-null      int64  
 2   created          6 non-null      int64  
 3   updated          6 non-null      int64  
 4   end_date_time    6 non-null      int64  
 5   start_date_time  6 non-null      int64  
 6   place_id         1 non-null      float64
 7   points           6 non-null      int64  
 8   redemptions      0 non-null      float64
dtypes: float64(2), int64(7)
memory usage: 560.0 bytes


In [ ]:
#dealing with missing data
df.drop(['redemptions','place_id'],axis=1,inplace=True)
#checking duplicates
df.drop_duplicates(subset=['id'],inplace=True)
#converting types (unix timestamps to datetime)
df['created'] = pd.to_datetime(df['created'],unit='s').dt.date
df['updated'] = pd.to_datetime(df['updated'],unit='s').dt.date
df['end_date_time'] = pd.to_datetime(df['end_date_time'],unit='s').dt.date
df['start_date_time'] = pd.to_datetime(df['start_date_time'],unit='s').dt.date
df['duration']=df['end_date_time']-df['start_date_time']

In [ ]:
df['duration']=df['end_date_time']-df['start_date_time']
df.head()

,id,user_id,created,updated,end_date_time,start_date_time,points,duration
0,61628,423,2021-03-05,2021-03-07,2021-03-09,2021-03-05,150,"4 days, 0:00:00"
1,61922,423,2021-03-12,2021-11-19,2021-12-30,2021-03-11,200,"294 days, 0:00:00"
2,61940,423,2021-03-12,2021-06-02,2021-12-31,2021-03-11,300,"295 days, 0:00:00"
3,63306,423,2021-03-31,2021-04-29,2021-12-31,2021-03-31,200,"275 days, 0:00:00"
4,63379,417,2021-04-01,2021-04-27,2021-12-31,2021-04-01,200,"274 days, 0:00:00"


In [ ]:
df.to_csv('D:/Deloitte/DIH-X-AUC-Hackathon/data/Cleaned Data/fct_bonus_codes.csv')

In [ ]:
#cleaning fct_orders.csv
data_loader = DataLoader("D:\Deloitte\DIH-X-AUC-Hackathon\data\Inventory Management")
df = data_loader.load_csv("fct_orders.csv")

D:\Deloitte\DIH-X-AUC-Hackathon\src\models\data_loader.py:56: DtypeWarning: Columns (10,17,18,26,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, parse_dates=parse_dates)


Successfully loaded fct_orders.csv: 399810 rows


In [ ]:
#knowing the shape, the columns, and the info of the dataset
print(df.shape)
print(df.columns.tolist())
df.head()

(399810, 40)
['id', 'user_id', 'created', 'updated', 'updated_by', 'account_id', 'cash_amount', 'cashier_notified', 'channel', 'customer_mobile_phone', 'customer_name', 'code', 'delivery_charge', 'delivery_location_id', 'demo_mode', 'discount_amount', 'driver_id', 'external_id', 'instructions', 'items_amount', 'payment_method', 'pickup_time', 'place_id', 'points_earned', 'points_redeemed', 'promise_time', 'rejection_reason', 'referring_user_id', 'service_charge', 'source', 'split_bill', 'split_bill_type', 'status', 'synchronized_to_accounting', 'table_id', 'tier_id', 'total_amount', 'trainee_mode', 'type', 'vat_amount']


,id,user_id,created,updated,updated_by,account_id,cash_amount,cashier_notified,channel,customer_mobile_phone,customer_name,code,delivery_charge,delivery_location_id,demo_mode,discount_amount,driver_id,external_id,instructions,items_amount,payment_method,pickup_time,place_id,points_earned,points_redeemed,promise_time,rejection_reason,referring_user_id,service_charge,source,split_bill,split_bill_type,status,synchronized_to_accounting,table_id,tier_id,total_amount,trainee_mode,type,vat_amount
0,60825,512,1613139429,1687470923,NaN,NaN,50.0,NaN,App,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,50.0,Counter,NaN,59897.0,25.0,0.0,1.613140e+09,NaN,NaN,NaN,Customer,NaN,NaN,Closed,NaN,NaN,59898.0,50.0,NaN,Takeaway,12.50
1,60841,514,1613144783,1687470923,NaN,NaN,655.0,NaN,App,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,655.0,Counter,NaN,59821.0,327.0,0.0,1.613146e+09,NaN,NaN,NaN,Customer,NaN,NaN,Closed,NaN,NaN,59822.0,655.0,NaN,Takeaway,163.75
2,60852,489,1613149610,1687470923,NaN,NaN,430.0,NaN,App,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,430.0,Counter,NaN,59821.0,215.0,0.0,1.613151e+09,NaN,NaN,NaN,Customer,NaN,NaN,Closed,NaN,NaN,59822.0,430.0,NaN,Takeaway,107.50
3,60863,522,1613149808,1687470923,NaN,NaN,495.0,NaN,App,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,495.0,Counter,NaN,59821.0,247.0,0.0,1.613151e+09,NaN,NaN,NaN,Customer,NaN,NaN,Closed,NaN,NaN,59822.0,495.0,NaN,Takeaway,123.75
4,60873,758,1613150426,1741152789,0.0,NaN,750.0,0.0,App,NaN,NaN,59821TH9VJY,0.0,NaN,0.0,0.0,NaN,NaN,NaN,750.0,Counter,NaN,59821.0,375.0,0.0,1.613153e+09,NaN,NaN,0.0,Customer,0.0,NaN,Closed,0.0,NaN,59822.0,750.0,0.0,Takeaway,187.50


In [ ]:
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399810 entries, 0 to 399809
Data columns (total 40 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   id                          399810 non-null  int64  
 1   user_id                     399810 non-null  int64  
 2   created                     399810 non-null  int64  
 3   updated                     399810 non-null  int64  
 4   updated_by                  1037 non-null    float64
 5   account_id                  2772 non-null    float64
 6   cash_amount                 399810 non-null  float64
 7   cashier_notified            399604 non-null  float64
 8   channel                     399810 non-null  object 
 9   customer_mobile_phone       1717 non-null    float64
 10  customer_name               49836 non-null   object 
 11  code                        399766 non-null  object 
 12  delivery_charge             399809 non-null  float64
 13  delivery_locat

In [ ]:
#dealing with missing data
#any missing id will be filled with zero or empty. some columns that their null means no information given like price will be removed
cols_to_drop = [
    'referring_user_id', 'driver_id', 'rejection_reason', 'split_bill_type', 
    'delivery_location_id', 'pickup_time', 'external_id', 'service_charge', 'customer_mobile_phone','customer_name', 'account_id', 'split_bill',
    'table_id', 'synchronized_to_accounting'
]
df.drop(cols_to_drop,axis=1,inplace=True)
df.loc[df['updated_by'].isnull(),'updated_by'] = df.loc[df['updated_by'].isnull(),'user_id'] #supposing that the null means the user updated them
df['instructions']=df['instructions'].fillna("none")
df.dropna(inplace=True)

In [ ]:
#converting unix timestamps to datetimes
df['created'] = pd.to_datetime(df['created'],unit='s').dt.date
df['updated'] = pd.to_datetime(df['updated'],unit='s').dt.date
df['updated_by'] = pd.to_datetime(df['updated_by'],unit='s').dt.date

In [ ]:
#removing outliers
cols_to_check = [col for col in df.select_dtypes(include=[np.number]).columns 
                 if not any(x in col.lower() for x in ['id', 'created', 'updated'])]
z_scores = np.abs(stats.zscore(df[cols_to_check]))    
df = df[(z_scores < 3).all(axis=1)]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 335214 entries, 23655 to 399808
Data columns (total 26 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                335214 non-null  int64  
 1   user_id           335214 non-null  int64  
 2   created           335214 non-null  object 
 3   updated           335214 non-null  object 
 4   updated_by        335214 non-null  object 
 5   cash_amount       335214 non-null  float64
 6   cashier_notified  335214 non-null  float64
 7   channel           335214 non-null  object 
 8   code              335214 non-null  object 
 9   delivery_charge   335214 non-null  float64
 10  demo_mode         335214 non-null  float64
 11  discount_amount   335214 non-null  float64
 12  instructions      335214 non-null  object 
 13  items_amount      335214 non-null  float64
 14  payment_method    335214 non-null  object 
 15  place_id          335214 non-null  float64
 16  points_earned     335

In [ ]:
df.head()

,id,user_id,created,updated,updated_by,cash_amount,cashier_notified,channel,code,delivery_charge,demo_mode,discount_amount,instructions,items_amount,payment_method,place_id,points_earned,points_redeemed,promise_time,source,status,tier_id,total_amount,trainee_mode,type,vat_amount
23655,187236,2510,2022-10-05,2025-03-05,1970-01-01,109.0,0.0,App,XMTKPW,0.0,0.0,0.0,none,109.0,Online,106493.0,109.0,0.0,1.664987e+09,Customer,Closed,106490.0,109.0,0.0,Takeaway,27.25
26278,197863,2953,2022-10-23,2025-03-05,1970-01-01,105.0,0.0,App,UTVHWJ,0.0,0.0,0.0,none,105.0,Online,106493.0,105.0,0.0,1.666542e+09,Customer,Closed,106490.0,105.0,0.0,Takeaway,26.25
32850,226397,0,2023-01-04,2024-11-06,1970-01-01,45.0,0.0,Web,94025TLCP7J,0.0,0.0,0.0,none,45.0,Card,94025.0,22.0,0.0,1.672841e+09,Customer,Rejected,94022.0,45.0,0.0,Eat In,11.25
32851,226399,0,2023-01-04,2024-11-06,1970-01-01,45.0,0.0,Web,94025E7MNKP,0.0,0.0,0.0,none,45.0,Card,94025.0,22.0,0.0,1.672842e+09,Customer,Rejected,94022.0,45.0,0.0,Eat In,11.25
32852,226401,0,2023-01-04,2024-11-06,1970-01-01,45.0,0.0,Web,940254R7QCN,0.0,0.0,0.0,none,45.0,Card,94025.0,22.0,0.0,1.672842e+09,Customer,Rejected,94022.0,45.0,0.0,Eat In,11.25


In [ ]:
df.to_csv('D:/Deloitte/DIH-X-AUC-Hackathon/data/Cleaned Data/fct_order.csv')

In [ ]:
#cleaning fct_invoice_items.csv
data_loader = DataLoader("D:\Deloitte\DIH-X-AUC-Hackathon\data\Inventory Management")
df = data_loader.load_csv("fct_invoice_items.csv")

Successfully loaded fct_invoice_items.csv: 3124 rows


In [ ]:
#knowing the shape, the columns, and the info of the dataset
print(df.shape)
print(df.columns.tolist())
df.head()

(3124, 8)
['id', 'user_id', 'created', 'updated', 'amount', 'product_id', 'description', 'invoice_id']


,id,user_id,created,updated,amount,product_id,description,invoice_id
0,321363,1,1689949801,1718715887,760.000,NaN,POS Light (76 days),321364.0
1,322423,1,1690193730,1718715887,206.667,NaN,POS Light (31 days),322424.0
2,323080,1,1690293901,1718715887,720.000,NaN,POS Light (72 days),323081.0
3,324088,1,1690413001,1718715887,710.000,NaN,POS Light (71 days),324089.0
4,324170,1,1690449002,1718715887,700.000,NaN,POS Light (70 days),324171.0


In [ ]:
#showing important info of the dataset
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3124 entries, 0 to 3123
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           3124 non-null   int64  
 1   user_id      3124 non-null   int64  
 2   created      3124 non-null   int64  
 3   updated      3124 non-null   int64  
 4   amount       3124 non-null   float64
 5   product_id   2937 non-null   float64
 6   description  3123 non-null   object 
 7   invoice_id   3105 non-null   float64
dtypes: float64(3), int64(4), object(1)
memory usage: 195.4+ KB


In [ ]:
df.dropna(inplace=True)

In [ ]:
df.to_csv('D:/Deloitte/DIH-X-AUC-Hackathon/data/Cleaned Data/fct_invoice_items.csv')

In [ ]:
#cleaning fct_cash_balances.csv
data_loader = DataLoader("D:\Deloitte\DIH-X-AUC-Hackathon\data\Inventory Management")
df = data_loader.load_csv("fct_cash_balances.csv")

Successfully loaded fct_cash_balances.csv: 52915 rows


In [ ]:
#knowing the shape, the columns, and the info of the dataset
print(df.shape)
print(df.columns.tolist())
df.head()

(52915, 12)
['id', 'user_id', 'created', 'updated', 'closing_coins_and_notes', 'closing_balance', 'end_time', 'place_id', 'opening_balance', 'opening_coins_and_notes', 'status', 'transactions']


,id,user_id,created,updated,closing_coins_and_notes,closing_balance,end_time,place_id,opening_balance,opening_coins_and_notes,status,transactions
0,420439,1,1698296813,1698468331,NaN,348.0,1.698468e+09,59897,150.0,NaN,Closed,"[{""created"":1698296818,""type"":""received"",""amou..."
1,421565,1,1698333424,1702299941,NaN,99602.5,1.702300e+09,316229,2000.0,NaN,Closed,"[{""created"":1698333427,""type"":""received"",""amou..."
2,421701,1,1698335712,1727107200,[],0.0,1.727107e+09,320584,200.0,[],Closed,"[{""created"":1727094293,""type"":""received"",""amou..."
3,421796,1,1698337033,1747045163,[],0.0,1.747045e+09,355205,500.0,[],Closed,"[{""created"":1700933961,""type"":""change"",""amount..."
4,422038,1,1698340777,1747045169,[],0.0,1.747045e+09,355205,132.0,[],Closed,"[{""created"":1700933961,""type"":""change"",""amount..."


In [ ]:
#showing important info of the dataset
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52915 entries, 0 to 52914
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       52915 non-null  int64  
 1   user_id                  52915 non-null  int64  
 2   created                  52915 non-null  int64  
 3   updated                  52915 non-null  int64  
 4   closing_coins_and_notes  49320 non-null  object 
 5   closing_balance          52788 non-null  float64
 6   end_time                 52474 non-null  float64
 7   place_id                 52915 non-null  int64  
 8   opening_balance          52915 non-null  float64
 9   opening_coins_and_notes  49281 non-null  object 
 10  status                   52915 non-null  object 
 11  transactions             52788 non-null  object 
dtypes: float64(3), int64(5), object(4)
memory usage: 4.8+ MB


In [ ]:
#cleaning missing data and convert from unix timestamps to datetime
df.dropna(inplace=True)
df['created'] = pd.to_datetime(df['created'],unit='s').dt.date
df['updated'] = pd.to_datetime(df['updated'],unit='s').dt.date

In [ ]:
df['transactions']

2        [{"created":1727094293,"type":"received","amou...
3        [{"created":1700933961,"type":"change","amount...
4        [{"created":1700933961,"type":"change","amount...
5        [{"created":1727192271,"type":"change","amount...
6        [{"created":1700933961,"type":"change","amount...
                               ...                        
52879    [{"created":1754837217,"type":"received","amou...
52880    [{"created":1754844919,"type":"received","amou...
52881    [{"created":1754829627,"type":"received","amou...
52898    [{"created":1754832621,"type":"received","amou...
52912                                                   []
Name: transactions, Length: 48939, dtype: object

In [ ]:
#feature creation for transactions
import ast
df['total_transactions_amount']= 0.0
df['total_transactions_type']= "in"
for index in df.index:
    value=0
    list = ast.literal_eval(df.loc[index, "transactions"]) if pd.notnull(df.loc[index, "transactions"]) else []
    for transaction in list:
        if transaction["type"]=="received":
            value+= transaction["amount"]
        else:
            value-= transaction["amount"]
    if value>=0:
        df.loc[index,"total_transactions_amount"] = value
        df.loc[index,"total_transactions_type"] = "recieved"
    else:
        df.loc[index,"total_transactions_amount"] = -1*value
        df.loc[index,"total_transactions_type"] = "changed"

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48939 entries, 2 to 52912
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         48939 non-null  int64  
 1   user_id                    48939 non-null  int64  
 2   created                    48939 non-null  object 
 3   updated                    48939 non-null  object 
 4   closing_coins_and_notes    48939 non-null  object 
 5   closing_balance            48939 non-null  float64
 6   end_time                   48939 non-null  float64
 7   place_id                   48939 non-null  int64  
 8   opening_balance            48939 non-null  float64
 9   opening_coins_and_notes    48939 non-null  object 
 10  status                     48939 non-null  object 
 11  transactions               48939 non-null  object 
 12  total_transactions_amount  48939 non-null  float64
 13  total_transactions_type    48939 non-null  object 


In [ ]:
df.head()

,id,user_id,created,updated,closing_coins_and_notes,closing_balance,end_time,place_id,opening_balance,opening_coins_and_notes,status,transactions,total_transactions_amount,total_transactions_type
2,421701,1,2023-10-26,2024-09-23,[],0.0,1.727107e+09,320584,200.0,[],Closed,"[{""created"":1727094293,""type"":""received"",""amou...",42333.00,recieved
3,421796,1,2023-10-26,2025-05-12,[],0.0,1.747045e+09,355205,500.0,[],Closed,"[{""created"":1700933961,""type"":""change"",""amount...",11742.00,recieved
4,422038,1,2023-10-26,2025-05-12,[],0.0,1.747045e+09,355205,132.0,[],Closed,"[{""created"":1700933961,""type"":""change"",""amount...",11298.00,recieved
5,423192,1,2023-10-27,2024-09-24,[],0.0,1.727206e+09,386681,700.0,[],Closed,"[{""created"":1727192271,""type"":""change"",""amount...",212266.05,recieved
6,423211,1,2023-10-27,2025-05-12,[],0.0,1.747045e+09,355205,21.0,[],Closed,"[{""created"":1700933961,""type"":""change"",""amount...",11162.00,recieved


In [ ]:
df.to_csv('D:/Deloitte/DIH-X-AUC-Hackathon/data/Cleaned Data/fct_cash_balances.csv')

In [ ]:
#knowing the shape, the columns, and the info of the dataset
print(df.shape)
print(df.columns.tolist())
df.head()

(1999341, 23)
['id', 'user_id', 'created', 'updated', 'title', 'campaign_id', 'commission_amount', 'cost', 'discount_amount', 'external_id', 'group', 'instructions', 'item_id', 'order_id', 'points_earned', 'points_redeemed', 'price', 'quantity', 'redemptions', 'removed_ingredients', 'add_on_ids', 'status', 'vat_amount']


,id,user_id,created,updated,title,campaign_id,commission_amount,cost,discount_amount,external_id,group,instructions,item_id,order_id,points_earned,points_redeemed,price,quantity,redemptions,removed_ingredients,add_on_ids,status,vat_amount
0,60824,512,1613139429,1708921636,Latte,NaN,1.50,50.0,0.0,NaN,NaN,NaN,60822,60825.0,NaN,NaN,25.0,2,0,NaN,NaN,Pending,NaN
1,60837,514,1613144783,1708921895,The Classic Meal,NaN,5.25,175.0,0.0,NaN,NaN,NaN,59999,60841.0,NaN,NaN,175.0,1,0,NaN,59995,Pending,NaN
2,60838,514,1613144783,1708921895,The Classic Meal,NaN,4.80,160.0,0.0,NaN,NaN,NaN,59999,60841.0,NaN,NaN,160.0,1,0,NaN,NaN,Pending,NaN
3,60839,514,1613144783,1708921895,The Classic Meal,NaN,4.80,160.0,0.0,NaN,NaN,NaN,59999,60841.0,NaN,NaN,160.0,1,0,NaN,NaN,Pending,NaN
4,60840,514,1613144783,1708921895,The Classic Meal,NaN,4.80,160.0,0.0,NaN,NaN,NaN,59999,60841.0,NaN,NaN,160.0,1,0,NaN,NaN,Pending,NaN


In [ ]:
#showing important info of the dataset
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1999341 entries, 0 to 1999340
Data columns (total 23 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   id                   1999341 non-null  int64  
 1   user_id              1999341 non-null  int64  
 2   created              1999341 non-null  int64  
 3   updated              1999341 non-null  int64  
 4   title                1999341 non-null  object 
 5   campaign_id          1941 non-null     float64
 6   commission_amount    1995417 non-null  float64
 7   cost                 1999341 non-null  float64
 8   discount_amount      1999341 non-null  float64
 9   external_id          4111 non-null     object 
 10  group                710286 non-null   float64
 11  instructions         2362 non-null     object 
 12  item_id              1999341 non-null  int64  
 13  order_id             1999179 non-null  float64
 14  points_earned        1990916 non-null  float64
 15

In [ ]:
#independednt cell to see each columns info (check missing values or 


In [ ]:
#dealing with missing data
#convert types
#any missing id will be filled with zero or empty. some columns that their null means no information given like price will be removed
df['campaign_id'] = df['campaign_id'].fillna('none')
df['external_id'] = df['external_id'].fillna('none')
df['removed_ingredients'] = df['removed_ingredients'].fillna('none')
df['add_on_ids'] = df['add_on_ids'].fillna('0')
df['order_id'] = df['order_id'].fillna(0)
df['commission_amount'] = df['commission_amount'].fillna(0)
df = df.dropna(subset=['price', 'status'])
df['group'] = df['group'].fillna(-1)
df['group'] = df['group'] + 1
df['instructions'] = df['instructions'].fillna('none')
df['points_earned'] = df['points_earned'].fillna(0)
df['points_redeemed'] = df['points_redeemed'].fillna(0)
df['vat_amount'] = df['vat_amount'].fillna(0)

In [ ]:
#checking duplicates
df.drop_duplicates(subset=['id'],inplace=True)

In [ ]:
#removing outliers
cols_to_check = [col for col in df.select_dtypes(include=[np.number]).columns 
                 if not any(x in col.lower() for x in ['id', 'created', 'updated'])]
z_scores = np.abs(stats.zscore(df[cols_to_check]))    
df = df[(z_scores < 2).all(axis=1)]

In [ ]:
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 1974592 entries, 0 to 1999339
Data columns (total 23 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   id                   1974592 non-null  int64  
 1   user_id              1974592 non-null  int64  
 2   created              1974592 non-null  int64  
 3   updated              1974592 non-null  int64  
 4   title                1974592 non-null  object 
 5   campaign_id          1974592 non-null  object 
 6   commission_amount    1974592 non-null  float64
 7   cost                 1974592 non-null  float64
 8   discount_amount      1974592 non-null  float64
 9   external_id          1974592 non-null  object 
 10  group                1974592 non-null  float64
 11  instructions         1974592 non-null  object 
 12  item_id              1974592 non-null  int64  
 13  order_id             1974592 non-null  float64
 14  points_earned        1974592 non-null  float64
 15  poi

In [ ]:
#converting types (unix timestamps to datetime)
df['created'] = pd.to_datetime(df['created'],unit='s').dt.date
df['updated'] = pd.to_datetime(df['updated'],unit='s').dt.date

In [ ]:
df.head()

,id,user_id,created,updated,title,campaign_id,commission_amount,cost,discount_amount,external_id,group,instructions,item_id,order_id,points_earned,points_redeemed,price,quantity,redemptions,removed_ingredients,add_on_ids,status,vat_amount
0,60824,512,2021-02-12,2024-02-26,Latte,none,1.50,50.0,0.0,none,0.0,none,60822,60825.0,0.0,0.0,25.0,2,0,none,0,Pending,0.0
1,60837,514,2021-02-12,2024-02-26,The Classic Meal,none,5.25,175.0,0.0,none,0.0,none,59999,60841.0,0.0,0.0,175.0,1,0,none,59995,Pending,0.0
2,60838,514,2021-02-12,2024-02-26,The Classic Meal,none,4.80,160.0,0.0,none,0.0,none,59999,60841.0,0.0,0.0,160.0,1,0,none,0,Pending,0.0
3,60839,514,2021-02-12,2024-02-26,The Classic Meal,none,4.80,160.0,0.0,none,0.0,none,59999,60841.0,0.0,0.0,160.0,1,0,none,0,Pending,0.0
4,60840,514,2021-02-12,2024-02-26,The Classic Meal,none,4.80,160.0,0.0,none,0.0,none,59999,60841.0,0.0,0.0,160.0,1,0,none,0,Pending,0.0


In [ ]:
df.to_csv('D:/Deloitte/DIH-X-AUC-Hackathon/data/Cleaned Data/fct_order_items.csv')

In [ ]:
#cleaning fct_bonus_codes.csv
data_loader = DataLoader("D:\Deloitte\DIH-X-AUC-Hackathon\data\Inventory Management")
df = data_loader.load_csv("fct_bonus_codes.csv")

Successfully loaded fct_bonus_codes.csv: 6 rows


In [ ]:
#knowing the shape, the columns, and the info of the dataset
print(df.shape)
print(df.columns.tolist())
df.head()

(6, 9)
['id', 'user_id', 'created', 'updated', 'end_date_time', 'start_date_time', 'place_id', 'points', 'redemptions']


,id,user_id,created,updated,end_date_time,start_date_time,place_id,points,redemptions
0,61628,423,1614960382,1615136541,1615248000,1614902400,NaN,150,NaN
1,61922,423,1615559087,1637324088,1640822400,1615420800,59897.0,200,NaN
2,61940,423,1615564762,1622661882,1640908800,1615420800,NaN,300,NaN
3,63306,423,1617184682,1619715679,1640908800,1617148800,NaN,200,NaN
4,63379,417,1617265478,1619514716,1640908800,1617235200,NaN,200,NaN


In [ ]:
#showing important info of the dataset
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               6 non-null      int64  
 1   user_id          6 non-null      int64  
 2   created          6 non-null      int64  
 3   updated          6 non-null      int64  
 4   end_date_time    6 non-null      int64  
 5   start_date_time  6 non-null      int64  
 6   place_id         1 non-null      float64
 7   points           6 non-null      int64  
 8   redemptions      0 non-null      float64
dtypes: float64(2), int64(7)
memory usage: 560.0 bytes


In [ ]:
#dealing with missing data
df.drop(['redemptions','place_id'],axis=1,inplace=True)
#checking duplicates
df.drop_duplicates(subset=['id'],inplace=True)
#converting types (unix timestamps to datetime)
df['created'] = pd.to_datetime(df['created'],unit='s').dt.date
df['updated'] = pd.to_datetime(df['updated'],unit='s').dt.date
df['end_date_time'] = pd.to_datetime(df['end_date_time'],unit='s').dt.date
df['start_date_time'] = pd.to_datetime(df['start_date_time'],unit='s').dt.date
df['duration']=df['end_date_time']-df['start_date_time']

In [ ]:
df['duration']=df['end_date_time']-df['start_date_time']
df.head()

,id,user_id,created,updated,end_date_time,start_date_time,points,duration
0,61628,423,2021-03-05,2021-03-07,2021-03-09,2021-03-05,150,"4 days, 0:00:00"
1,61922,423,2021-03-12,2021-11-19,2021-12-30,2021-03-11,200,"294 days, 0:00:00"
2,61940,423,2021-03-12,2021-06-02,2021-12-31,2021-03-11,300,"295 days, 0:00:00"
3,63306,423,2021-03-31,2021-04-29,2021-12-31,2021-03-31,200,"275 days, 0:00:00"
4,63379,417,2021-04-01,2021-04-27,2021-12-31,2021-04-01,200,"274 days, 0:00:00"


In [ ]:
df.to_csv('D:/Deloitte/DIH-X-AUC-Hackathon/data/Cleaned Data/fct_bonus_codes.csv')

In [ ]:
#cleaning fct_orders.csv
data_loader = DataLoader("D:\Deloitte\DIH-X-AUC-Hackathon\data\Inventory Management")
df = data_loader.load_csv("fct_orders.csv")

D:\Deloitte\DIH-X-AUC-Hackathon\src\models\data_loader.py:56: DtypeWarning: Columns (10,17,18,26,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, parse_dates=parse_dates)


Successfully loaded fct_orders.csv: 399810 rows


In [ ]:
#knowing the shape, the columns, and the info of the dataset
print(df.shape)
print(df.columns.tolist())
df.head()

(399810, 40)
['id', 'user_id', 'created', 'updated', 'updated_by', 'account_id', 'cash_amount', 'cashier_notified', 'channel', 'customer_mobile_phone', 'customer_name', 'code', 'delivery_charge', 'delivery_location_id', 'demo_mode', 'discount_amount', 'driver_id', 'external_id', 'instructions', 'items_amount', 'payment_method', 'pickup_time', 'place_id', 'points_earned', 'points_redeemed', 'promise_time', 'rejection_reason', 'referring_user_id', 'service_charge', 'source', 'split_bill', 'split_bill_type', 'status', 'synchronized_to_accounting', 'table_id', 'tier_id', 'total_amount', 'trainee_mode', 'type', 'vat_amount']


,id,user_id,created,updated,updated_by,account_id,cash_amount,cashier_notified,channel,customer_mobile_phone,customer_name,code,delivery_charge,delivery_location_id,demo_mode,discount_amount,driver_id,external_id,instructions,items_amount,payment_method,pickup_time,place_id,points_earned,points_redeemed,promise_time,rejection_reason,referring_user_id,service_charge,source,split_bill,split_bill_type,status,synchronized_to_accounting,table_id,tier_id,total_amount,trainee_mode,type,vat_amount
0,60825,512,1613139429,1687470923,NaN,NaN,50.0,NaN,App,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,50.0,Counter,NaN,59897.0,25.0,0.0,1.613140e+09,NaN,NaN,NaN,Customer,NaN,NaN,Closed,NaN,NaN,59898.0,50.0,NaN,Takeaway,12.50
1,60841,514,1613144783,1687470923,NaN,NaN,655.0,NaN,App,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,655.0,Counter,NaN,59821.0,327.0,0.0,1.613146e+09,NaN,NaN,NaN,Customer,NaN,NaN,Closed,NaN,NaN,59822.0,655.0,NaN,Takeaway,163.75
2,60852,489,1613149610,1687470923,NaN,NaN,430.0,NaN,App,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,430.0,Counter,NaN,59821.0,215.0,0.0,1.613151e+09,NaN,NaN,NaN,Customer,NaN,NaN,Closed,NaN,NaN,59822.0,430.0,NaN,Takeaway,107.50
3,60863,522,1613149808,1687470923,NaN,NaN,495.0,NaN,App,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,495.0,Counter,NaN,59821.0,247.0,0.0,1.613151e+09,NaN,NaN,NaN,Customer,NaN,NaN,Closed,NaN,NaN,59822.0,495.0,NaN,Takeaway,123.75
4,60873,758,1613150426,1741152789,0.0,NaN,750.0,0.0,App,NaN,NaN,59821TH9VJY,0.0,NaN,0.0,0.0,NaN,NaN,NaN,750.0,Counter,NaN,59821.0,375.0,0.0,1.613153e+09,NaN,NaN,0.0,Customer,0.0,NaN,Closed,0.0,NaN,59822.0,750.0,0.0,Takeaway,187.50


In [ ]:
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399810 entries, 0 to 399809
Data columns (total 40 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   id                          399810 non-null  int64  
 1   user_id                     399810 non-null  int64  
 2   created                     399810 non-null  int64  
 3   updated                     399810 non-null  int64  
 4   updated_by                  1037 non-null    float64
 5   account_id                  2772 non-null    float64
 6   cash_amount                 399810 non-null  float64
 7   cashier_notified            399604 non-null  float64
 8   channel                     399810 non-null  object 
 9   customer_mobile_phone       1717 non-null    float64
 10  customer_name               49836 non-null   object 
 11  code                        399766 non-null  object 
 12  delivery_charge             399809 non-null  float64
 13  delivery_locat

In [ ]:
#dealing with missing data
#any missing id will be filled with zero or empty. some columns that their null means no information given like price will be removed
cols_to_drop = [
    'referring_user_id', 'driver_id', 'rejection_reason', 'split_bill_type', 
    'delivery_location_id', 'pickup_time', 'external_id', 'service_charge', 'customer_mobile_phone','customer_name', 'account_id', 'split_bill',
    'table_id', 'synchronized_to_accounting'
]
df.drop(cols_to_drop,axis=1,inplace=True)
df.loc[df['updated_by'].isnull(),'updated_by'] = df.loc[df['updated_by'].isnull(),'user_id'] #supposing that the null means the user updated them
df['instructions']=df['instructions'].fillna("none")
df.dropna(inplace=True)

In [ ]:
#converting unix timestamps to datetimes
df['created'] = pd.to_datetime(df['created'],unit='s').dt.date
df['updated'] = pd.to_datetime(df['updated'],unit='s').dt.date
df['updated_by'] = pd.to_datetime(df['updated_by'],unit='s').dt.date

In [ ]:
#removing outliers
cols_to_check = [col for col in df.select_dtypes(include=[np.number]).columns 
                 if not any(x in col.lower() for x in ['id', 'created', 'updated'])]
z_scores = np.abs(stats.zscore(df[cols_to_check]))    
df = df[(z_scores < 3).all(axis=1)]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 335214 entries, 23655 to 399808
Data columns (total 26 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                335214 non-null  int64  
 1   user_id           335214 non-null  int64  
 2   created           335214 non-null  object 
 3   updated           335214 non-null  object 
 4   updated_by        335214 non-null  object 
 5   cash_amount       335214 non-null  float64
 6   cashier_notified  335214 non-null  float64
 7   channel           335214 non-null  object 
 8   code              335214 non-null  object 
 9   delivery_charge   335214 non-null  float64
 10  demo_mode         335214 non-null  float64
 11  discount_amount   335214 non-null  float64
 12  instructions      335214 non-null  object 
 13  items_amount      335214 non-null  float64
 14  payment_method    335214 non-null  object 
 15  place_id          335214 non-null  float64
 16  points_earned     335

In [ ]:
df.head()

,id,user_id,created,updated,updated_by,cash_amount,cashier_notified,channel,code,delivery_charge,demo_mode,discount_amount,instructions,items_amount,payment_method,place_id,points_earned,points_redeemed,promise_time,source,status,tier_id,total_amount,trainee_mode,type,vat_amount
23655,187236,2510,2022-10-05,2025-03-05,1970-01-01,109.0,0.0,App,XMTKPW,0.0,0.0,0.0,none,109.0,Online,106493.0,109.0,0.0,1.664987e+09,Customer,Closed,106490.0,109.0,0.0,Takeaway,27.25
26278,197863,2953,2022-10-23,2025-03-05,1970-01-01,105.0,0.0,App,UTVHWJ,0.0,0.0,0.0,none,105.0,Online,106493.0,105.0,0.0,1.666542e+09,Customer,Closed,106490.0,105.0,0.0,Takeaway,26.25
32850,226397,0,2023-01-04,2024-11-06,1970-01-01,45.0,0.0,Web,94025TLCP7J,0.0,0.0,0.0,none,45.0,Card,94025.0,22.0,0.0,1.672841e+09,Customer,Rejected,94022.0,45.0,0.0,Eat In,11.25
32851,226399,0,2023-01-04,2024-11-06,1970-01-01,45.0,0.0,Web,94025E7MNKP,0.0,0.0,0.0,none,45.0,Card,94025.0,22.0,0.0,1.672842e+09,Customer,Rejected,94022.0,45.0,0.0,Eat In,11.25
32852,226401,0,2023-01-04,2024-11-06,1970-01-01,45.0,0.0,Web,940254R7QCN,0.0,0.0,0.0,none,45.0,Card,94025.0,22.0,0.0,1.672842e+09,Customer,Rejected,94022.0,45.0,0.0,Eat In,11.25


In [ ]:
df.to_csv('D:/Deloitte/DIH-X-AUC-Hackathon/data/Cleaned Data/fct_order.csv')

In [ ]:
#cleaning fct_invoice_items.csv
data_loader = DataLoader("D:\Deloitte\DIH-X-AUC-Hackathon\data\Inventory Management")
df = data_loader.load_csv("fct_invoice_items.csv")

Successfully loaded fct_invoice_items.csv: 3124 rows


In [ ]:
#knowing the shape, the columns, and the info of the dataset
print(df.shape)
print(df.columns.tolist())
df.head()

(3124, 8)
['id', 'user_id', 'created', 'updated', 'amount', 'product_id', 'description', 'invoice_id']


,id,user_id,created,updated,amount,product_id,description,invoice_id
0,321363,1,1689949801,1718715887,760.000,NaN,POS Light (76 days),321364.0
1,322423,1,1690193730,1718715887,206.667,NaN,POS Light (31 days),322424.0
2,323080,1,1690293901,1718715887,720.000,NaN,POS Light (72 days),323081.0
3,324088,1,1690413001,1718715887,710.000,NaN,POS Light (71 days),324089.0
4,324170,1,1690449002,1718715887,700.000,NaN,POS Light (70 days),324171.0


In [ ]:
#showing important info of the dataset
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3124 entries, 0 to 3123
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           3124 non-null   int64  
 1   user_id      3124 non-null   int64  
 2   created      3124 non-null   int64  
 3   updated      3124 non-null   int64  
 4   amount       3124 non-null   float64
 5   product_id   2937 non-null   float64
 6   description  3123 non-null   object 
 7   invoice_id   3105 non-null   float64
dtypes: float64(3), int64(4), object(1)
memory usage: 195.4+ KB


In [ ]:
df.dropna(inplace=True)

In [ ]:
df.to_csv('D:/Deloitte/DIH-X-AUC-Hackathon/data/Cleaned Data/fct_invoice_items.csv')

In [ ]:
#cleaning fct_cash_balances.csv
data_loader = DataLoader("D:\Deloitte\DIH-X-AUC-Hackathon\data\Inventory Management")
df = data_loader.load_csv("fct_cash_balances.csv")

Successfully loaded fct_cash_balances.csv: 52915 rows


In [ ]:
#knowing the shape, the columns, and the info of the dataset
print(df.shape)
print(df.columns.tolist())
df.head()

(52915, 12)
['id', 'user_id', 'created', 'updated', 'closing_coins_and_notes', 'closing_balance', 'end_time', 'place_id', 'opening_balance', 'opening_coins_and_notes', 'status', 'transactions']


,id,user_id,created,updated,closing_coins_and_notes,closing_balance,end_time,place_id,opening_balance,opening_coins_and_notes,status,transactions
0,420439,1,1698296813,1698468331,NaN,348.0,1.698468e+09,59897,150.0,NaN,Closed,"[{""created"":1698296818,""type"":""received"",""amou..."
1,421565,1,1698333424,1702299941,NaN,99602.5,1.702300e+09,316229,2000.0,NaN,Closed,"[{""created"":1698333427,""type"":""received"",""amou..."
2,421701,1,1698335712,1727107200,[],0.0,1.727107e+09,320584,200.0,[],Closed,"[{""created"":1727094293,""type"":""received"",""amou..."
3,421796,1,1698337033,1747045163,[],0.0,1.747045e+09,355205,500.0,[],Closed,"[{""created"":1700933961,""type"":""change"",""amount..."
4,422038,1,1698340777,1747045169,[],0.0,1.747045e+09,355205,132.0,[],Closed,"[{""created"":1700933961,""type"":""change"",""amount..."


In [ ]:
#showing important info of the dataset
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52915 entries, 0 to 52914
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       52915 non-null  int64  
 1   user_id                  52915 non-null  int64  
 2   created                  52915 non-null  int64  
 3   updated                  52915 non-null  int64  
 4   closing_coins_and_notes  49320 non-null  object 
 5   closing_balance          52788 non-null  float64
 6   end_time                 52474 non-null  float64
 7   place_id                 52915 non-null  int64  
 8   opening_balance          52915 non-null  float64
 9   opening_coins_and_notes  49281 non-null  object 
 10  status                   52915 non-null  object 
 11  transactions             52788 non-null  object 
dtypes: float64(3), int64(5), object(4)
memory usage: 4.8+ MB


In [ ]:
#cleaning missing data and convert from unix timestamps to datetime
df.dropna(inplace=True)
df['created'] = pd.to_datetime(df['created'],unit='s').dt.date
df['updated'] = pd.to_datetime(df['updated'],unit='s').dt.date

In [ ]:
df['transactions']

2        [{"created":1727094293,"type":"received","amou...
3        [{"created":1700933961,"type":"change","amount...
4        [{"created":1700933961,"type":"change","amount...
5        [{"created":1727192271,"type":"change","amount...
6        [{"created":1700933961,"type":"change","amount...
                               ...                        
52879    [{"created":1754837217,"type":"received","amou...
52880    [{"created":1754844919,"type":"received","amou...
52881    [{"created":1754829627,"type":"received","amou...
52898    [{"created":1754832621,"type":"received","amou...
52912                                                   []
Name: transactions, Length: 48939, dtype: object

In [ ]:
#feature creation for transactions
import ast
df['total_transactions_amount']= 0.0
df['total_transactions_type']= "in"
for index in df.index:
    value=0
    list = ast.literal_eval(df.loc[index, "transactions"]) if pd.notnull(df.loc[index, "transactions"]) else []
    for transaction in list:
        if transaction["type"]=="received":
            value+= transaction["amount"]
        else:
            value-= transaction["amount"]
    if value>=0:
        df.loc[index,"total_transactions_amount"] = value
        df.loc[index,"total_transactions_type"] = "recieved"
    else:
        df.loc[index,"total_transactions_amount"] = -1*value
        df.loc[index,"total_transactions_type"] = "changed"

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48939 entries, 2 to 52912
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         48939 non-null  int64  
 1   user_id                    48939 non-null  int64  
 2   created                    48939 non-null  object 
 3   updated                    48939 non-null  object 
 4   closing_coins_and_notes    48939 non-null  object 
 5   closing_balance            48939 non-null  float64
 6   end_time                   48939 non-null  float64
 7   place_id                   48939 non-null  int64  
 8   opening_balance            48939 non-null  float64
 9   opening_coins_and_notes    48939 non-null  object 
 10  status                     48939 non-null  object 
 11  transactions               48939 non-null  object 
 12  total_transactions_amount  48939 non-null  float64
 13  total_transactions_type    48939 non-null  object 


In [ ]:
df.head()

,id,user_id,created,updated,closing_coins_and_notes,closing_balance,end_time,place_id,opening_balance,opening_coins_and_notes,status,transactions,total_transactions_amount,total_transactions_type
2,421701,1,2023-10-26,2024-09-23,[],0.0,1.727107e+09,320584,200.0,[],Closed,"[{""created"":1727094293,""type"":""received"",""amou...",42333.00,recieved
3,421796,1,2023-10-26,2025-05-12,[],0.0,1.747045e+09,355205,500.0,[],Closed,"[{""created"":1700933961,""type"":""change"",""amount...",11742.00,recieved
4,422038,1,2023-10-26,2025-05-12,[],0.0,1.747045e+09,355205,132.0,[],Closed,"[{""created"":1700933961,""type"":""change"",""amount...",11298.00,recieved
5,423192,1,2023-10-27,2024-09-24,[],0.0,1.727206e+09,386681,700.0,[],Closed,"[{""created"":1727192271,""type"":""change"",""amount...",212266.05,recieved
6,423211,1,2023-10-27,2025-05-12,[],0.0,1.747045e+09,355205,21.0,[],Closed,"[{""created"":1700933961,""type"":""change"",""amount...",11162.00,recieved


In [ ]:
df.to_csv('D:/Deloitte/DIH-X-AUC-Hackathon/data/Cleaned Data/fct_cash_balances.csv')

In [73]:
#cleaning fct_order_items.csv
data_loader = DataLoader("D:\Deloitte\DIH-X-AUC-Hackathon\data\Inventory Management")
df = data_loader.load_csv("dim_places.csv")

Successfully loaded dim_places.csv: 1824 rows


In [74]:
#knowing the shape, the columns, and the info of the dataset
print(df.shape)
print(df.columns.tolist())
df.head()

(1824, 189)
['id', 'user_id', 'created', 'updated', 'title', 'activated', 'active', 'bankrupt', 'binding_period', 'chain_id', 'contact_email', 'contact_mobile_phone', 'contract_start', 'country', 'currency', 'cvr_number', 'duplicate', 'ecommerce_fee', 'eu_commission', 'invoicing_start_date', 'isv_commission', 'isv_partner', 'non_eu_commission', 'onboarded_by', 'payment_terminal_provider', 'area_id', 'processing_fee', 'sales_outcome_id', 'sales_stage', 'service_charge', 'setup_type', 'termination_date', 'termination_value', 'transferred_contract', 'type_id', 'cashier_ids', 'manager_ids', 'driver_ids', 'payment_terminal_ids', 'kitchen_staff_ids', 'contact_name', 'description', 'cuisine_ids', 'trainee_mode', 'demo_mode', 'default_order_type_cashier', 'default_order_type_customer', 'show_bestsellers', 'use_quick_search', 'display_cashier_images', 'display_created_by', 'use_customer_facing_display', 'timezone', 'contact_language', 'business_name', 'area', 'street_address', 'suppress_receipt

,id,user_id,created,updated,title,activated,active,bankrupt,binding_period,chain_id,contact_email,contact_mobile_phone,contract_start,country,currency,cvr_number,duplicate,ecommerce_fee,eu_commission,invoicing_start_date,isv_commission,isv_partner,non_eu_commission,onboarded_by,payment_terminal_provider,area_id,processing_fee,sales_outcome_id,sales_stage,service_charge,setup_type,termination_date,termination_value,transferred_contract,type_id,cashier_ids,manager_ids,driver_ids,payment_terminal_ids,kitchen_staff_ids,contact_name,description,cuisine_ids,trainee_mode,demo_mode,default_order_type_cashier,default_order_type_customer,show_bestsellers,use_quick_search,display_cashier_images,display_created_by,use_customer_facing_display,timezone,contact_language,business_name,area,street_address,suppress_receipt_prompt,print_on_acceptance,enable_receipt_download,enable_email_receipts,customer_receipt_cc,customer_receipt_msg,daily_sales_reports,monthly_sales_reports,sales_report_email,logo,image_1,image_2,image_3,image_4,image_5,phone,email,website,facebook,instagram,takeaway_link,delivery_link,table_booking_link,opening_hours,enable_cashback,enable_voucher_payment,inventory_management,seasonal,moved_to_competition,dormant,dormant_partner,dead_lead,closed,collect_customer_info_takeaway,collect_customer_info_eat_in,collect_customer_info_delivery,nets_mid,realm,auto_accept_delivery,auto_accept_eat_in,auto_accept_table_bookings,auto_accept_takeaway,auto_close_delivery,auto_close_eat_in,auto_close_takeaway,auto_hide_orders_eat_in,auto_sms_topup,average_driver_speed,call_specific_waiter,call_waiter,call_waiter_interval,cash_pay_delivery,cash_pay_eat_in,cash_pay_takeaway,cashier_homepage,accepting_orders,allow_future_pickup_time,allow_refunds,default_payment_method,delivery,delivery_qr_code_link,delivery_radius,demo_end,display_cashier_prices,display_in_app,display_item_ratings,display_order_status,dynamic_location,eat_in,ecom_provider,enable_barcode_scanning,enable_cash_in_out,enable_ecommerce_payments,enable_nfc_login,enable_ordered_printing,enable_other_payment,enable_qr_code_payment,enable_qr_code_receipt,enable_sms_on_ready,first_day_of_week,force_table_number,force_upfront_pay_delivery,force_upfront_pay_eat_in,force_upfront_pay_takeaway,immediate_payment_delivery,immediate_payment_eat_in,immediate_payment_takeaway,in_app_pay_delivery,in_app_pay_eat_in,in_app_pay_takeaway,isv_source_code,language,latitude,longitude,loyalty_program,min_guests,minimum_delivery_amount,not_accepting_orders_msg,payment_terminal_pay_delivery,payment_terminal_pay_eat_in,payment_terminal_pay_takeaway,print_on_payment,print_on_refund,print_sold_items,prompt_for_tip_delivery,prompt_for_tip_eat_in,prompt_for_tip_takeaway,protect_sensitive_sections,rating,receipt_printing_enabled,receive_phone_calls,skip_order_confirmation,slot_duration,sms_balance,sms_on_ready_text,suppress_print_receipt_prompt,table_bookings,takeaway,takeaway_qr_code_link,terminal_provider,theme,tiered_loyalty_program,vat,vip_threshold,visit_duration,votes,waiting_time,unspecified_section_id,unspecified_item_id,enable_sms_on_order,sms_on_order_text,exclude_from_ops_report
0,59813,417,1610718144,1744221291,Green Love,0,0,0,36,NaN,place_59813@example.com,4559813,NaN,DK,DKK,30025334,0,0.0,0.0,0,2.0,0,0.0,lovingloyalty,NaN,11434.0,0.0,NaN,Shop info filled,0.0,NaN,0,NaN,0,1335.0,424|551,423|551|424,NaN,NaN,NaN,Mr./Mrs. 59813,Green Love er et plantebaseret vegansk spisest...,12179,0,0,Takeaway,Takeaway,1,0,1,0,0,Europe/Copenhagen,da,NaN,2000 Frederiksberg,Godthåbsvej 142,1,0,0,0,NaN,NaN,1,1,NaN,NaN,https://storage.googleapis.com/loving-loyalty-...,https://storage.googleapis.com/loving-loyalty-...,https://storage.googleapis.com/loving-loyalty-...,https://storage.googleapis.com/loving-loyalty-...,NaN,4559813,place_59813@example.com,https://greenlove.dk/,https://www.facebook.com/Green-Love-Denmark-10...,https://www.instagram.com/green_love_denmark/,https://lovingloyalty.page.link/ayw8rvzr1AeGcBtH9,https:

In [75]:
#showing important info of the dataset
pd.set_option('display.max_columns', None)
pd.set_option('display.max_seq_items', None)
pd.set_option('display.max_info_columns', 200)
print(df.info(max_cols=200))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1824 entries, 0 to 1823
Data columns (total 189 columns):
 #    Column                          Non-Null Count  Dtype  
---   ------                          --------------  -----  
 0    id                              1824 non-null   int64  
 1    user_id                         1824 non-null   int64  
 2    created                         1824 non-null   int64  
 3    updated                         1824 non-null   int64  
 4    title                           1824 non-null   object 
 5    activated                       1824 non-null   int64  
 6    active                          1824 non-null   int64  
 7    bankrupt                        1824 non-null   int64  
 8    binding_period                  1824 non-null   int64  
 9    chain_id                        2 non-null      float64
 10   contact_email                   1824 non-null   object 
 11   contact_mobile_phone            1824 non-null   int64  
 12   contract_start    

In [69]:
#independednt cell to see each columns info (check missing values or 
df['display_item_ratings'].unique()

array([0, 1], dtype=int64)

In [78]:
#dealing with missing data
#convert types
#any missing id will be filled with zero or empty. some columns that their null means no information given like price will be removed
import numpy as np
from scipy import stats
df =df.query("duplicate==0")
high_quality_cols = [
    'id', 'user_id', 'created', 'updated', 'title', 'activated', 'active', 
    'bankrupt', 'binding_period', 'contact_email', 'contact_mobile_phone', 
    'country', 'currency', 'duplicate', 'ecommerce_fee', 'eu_commission', 
    'invoicing_start_date', 'isv_commission', 'isv_partner', 'non_eu_commission', 
    'onboarded_by', 'processing_fee', 'sales_stage', 'service_charge', 
    'termination_date', 'transferred_contract', 'type_id', 'contact_name', 
    'cuisine_ids', 'trainee_mode', 'demo_mode', 'default_order_type_cashier', 
    'default_order_type_customer', 'show_bestsellers', 'use_quick_search', 
    'display_cashier_images', 'display_created_by', 'use_customer_facing_display', 
    'timezone', 'contact_language', 'suppress_receipt_prompt', 
    'print_on_acceptance', 'enable_receipt_download', 'enable_email_receipts', 
    'daily_sales_reports', 'monthly_sales_reports', 'phone', 'email', 
    'takeaway_link', 'delivery_link', 'opening_hours', 'enable_cashback', 
    'enable_voucher_payment', 'inventory_management', 'seasonal', 
    'moved_to_competition', 'dormant', 'dormant_partner', 'dead_lead', 
    'closed', 'collect_customer_info_takeaway', 'collect_customer_info_eat_in', 
    'collect_customer_info_delivery', 'auto_accept_delivery', 
    'auto_accept_eat_in', 'auto_accept_table_bookings', 'auto_accept_takeaway', 
    'auto_close_delivery', 'auto_close_eat_in', 'auto_close_takeaway', 
    'auto_hide_orders_eat_in', 'auto_sms_topup', 'average_driver_speed', 
    'call_specific_waiter', 'call_waiter', 'call_waiter_interval', 
    'cash_pay_delivery', 'cash_pay_eat_in', 'cash_pay_takeaway', 
    'cashier_homepage', 'accepting_orders', 'allow_future_pickup_time', 
    'allow_refunds', 'default_payment_method', 'delivery', 'delivery_radius', 
    'display_cashier_prices', 'display_in_app', 'display_item_ratings', 
    'display_order_status', 'dynamic_location', 'eat_in', 
    'enable_barcode_scanning', 'enable_cash_in_out', 'enable_ecommerce_payments', 
    'enable_nfc_login', 'enable_ordered_printing', 'enable_other_payment', 
    'enable_qr_code_payment', 'enable_qr_code_receipt', 'enable_sms_on_ready', 
    'first_day_of_week', 'force_table_number', 'force_upfront_pay_delivery', 
    'force_upfront_pay_eat_in', 'force_upfront_pay_takeaway', 
    'immediate_payment_delivery', 'immediate_payment_eat_in', 
    'immediate_payment_takeaway', 'in_app_pay_delivery', 'in_app_pay_eat_in', 
    'in_app_pay_takeaway', 'language', 'loyalty_program', 'min_guests', 
    'minimum_delivery_amount', 'payment_terminal_pay_delivery', 
    'payment_terminal_pay_eat_in', 'payment_terminal_pay_takeaway', 
    'print_on_payment', 'print_on_refund', 'print_sold_items', 
    'prompt_for_tip_delivery', 'prompt_for_tip_eat_in', 
    'prompt_for_tip_takeaway', 'protect_sensitive_sections', 'rating', 
    'receipt_printing_enabled', 'receive_phone_calls', 'skip_order_confirmation', 
    'slot_duration', 'sms_balance', 'suppress_print_receipt_prompt', 
    'table_bookings', 'takeaway', 'tiered_loyalty_program', 'vat', 
    'vip_threshold', 'visit_duration', 'votes', 'waiting_time', 
    'unspecified_section_id', 'unspecified_item_id', 'enable_sms_on_order', 
    'exclude_from_ops_report'
]
important_fillable_cols = [
    'business_name',          
    'area'                    
]
high_quality_cols = high_quality_cols + important_fillable_cols
df = df[high_quality_cols]
df['business_name']=df['business_name'].fillna("not_given")
df['area']=df['area'].fillna("not_given")
df['created'] = pd.to_datetime(df['created'],unit='s').dt.date
df['updated'] = pd.to_datetime(df['updated'],unit='s').dt.date
df['invoicing_start_date'] = pd.to_datetime(df['invoicing_start_date'],unit='s').dt.date
df.dropna(inplace=True)


ValueError: unit='s' not valid with non-numerical val='2021-01-15', at position 0

In [90]:
cols_to_check = [col for col in df.select_dtypes(include=[np.number]).columns 
                if not any(x in col.lower() for x in ['id', 'created', 'updated'])]
z_scores = np.abs(stats.zscore(df[cols_to_check]))    
df = df[~(z_scores >= 3).any(axis=1)]

In [91]:
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 1056 entries, 158 to 1823
Data columns (total 147 columns):
 #    Column                          Non-Null Count  Dtype  
---   ------                          --------------  -----  
 0    id                              1056 non-null   int64  
 1    user_id                         1056 non-null   int64  
 2    created                         1056 non-null   object 
 3    updated                         1056 non-null   object 
 4    title                           1056 non-null   object 
 5    activated                       1056 non-null   int64  
 6    active                          1056 non-null   int64  
 7    bankrupt                        1056 non-null   int64  
 8    binding_period                  1056 non-null   int64  
 9    contact_email                   1056 non-null   object 
 10   contact_mobile_phone            1056 non-null   int64  
 11   country                         1056 non-null   object 
 12   currency             

In [94]:
df.head()

,id,user_id,created,updated,title,activated,active,bankrupt,binding_period,contact_email,contact_mobile_phone,country,currency,duplicate,ecommerce_fee,eu_commission,invoicing_start_date,isv_commission,isv_partner,non_eu_commission,onboarded_by,processing_fee,sales_stage,service_charge,termination_date,transferred_contract,type_id,contact_name,cuisine_ids,trainee_mode,demo_mode,default_order_type_cashier,default_order_type_customer,show_bestsellers,use_quick_search,display_cashier_images,display_created_by,use_customer_facing_display,timezone,contact_language,suppress_receipt_prompt,print_on_acceptance,enable_receipt_download,enable_email_receipts,daily_sales_reports,monthly_sales_reports,phone,email,takeaway_link,delivery_link,opening_hours,enable_cashback,enable_voucher_payment,inventory_management,seasonal,moved_to_competition,dormant,dormant_partner,dead_lead,closed,collect_customer_info_takeaway,collect_customer_info_eat_in,collect_customer_info_delivery,auto_accept_delivery,auto_accept_eat_in,auto_accept_table_bookings,auto_accept_takeaway,auto_close_delivery,auto_close_eat_in,auto_close_takeaway,auto_hide_orders_eat_in,auto_sms_topup,average_driver_speed,call_specific_waiter,call_waiter,call_waiter_interval,cash_pay_delivery,cash_pay_eat_in,cash_pay_takeaway,cashier_homepage,accepting_orders,allow_future_pickup_time,allow_refunds,default_payment_method,delivery,delivery_radius,display_cashier_prices,display_in_app,display_item_ratings,display_order_status,dynamic_location,eat_in,enable_barcode_scanning,enable_cash_in_out,enable_ecommerce_payments,enable_nfc_login,enable_ordered_printing,enable_other_payment,enable_qr_code_payment,enable_qr_code_receipt,enable_sms_on_ready,first_day_of_week,force_table_number,force_upfront_pay_delivery,force_upfront_pay_eat_in,force_upfront_pay_takeaway,immediate_payment_delivery,immediate_payment_eat_in,immediate_payment_takeaway,in_app_pay_delivery,in_app_pay_eat_in,in_app_pay_takeaway,language,loyalty_program,min_guests,minimum_delivery_amount,payment_terminal_pay_delivery,payment_terminal_pay_eat_in,payment_terminal_pay_takeaway,print_on_payment,print_on_refund,print_sold_items,prompt_for_tip_delivery,prompt_for_tip_eat_in,prompt_for_tip_takeaway,protect_sensitive_sections,rating,receipt_printing_enabled,receive_phone_calls,skip_order_confirmation,slot_duration,sms_balance,suppress_print_receipt_prompt,table_bookings,takeaway,tiered_loyalty_program,vat,vip_threshold,visit_duration,votes,waiting_time,unspecified_section_id,unspecified_item_id,enable_sms_on_order,exclude_from_ops_report,business_name,area
158,354571,510,2023-09-05,2025-08-05,Bogø Aps,0,0,0,36,place_354571@example.com,45354571,DK,DKK,0,0.0,0.0,1970-01-01,0.0,0,0.0,lovingloyalty,0.0,Demo created,0.0,0,0,12191.0,Mr./Mrs. 354571,12192,0,0,Takeaway,Takeaway,1,1,0,0,0,Europe/Copenhagen,da,1,0,0,0,1,1,45354571,place_354571@example.com,https://lovingloyalty.page.link/7V98AUkbCTg9imYy6,https://lovingloyalty.page.link/Phsgd1ggbk6jfkC26,"{""monday"":{""from"":""closed"",""to"":""00.00""},""tues...",0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,1,0,1,40,0,1,60,0,1,1,menu,1,1,1,payment_terminal,0,5.0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,da,0,1,0.0,0,1,1,0,1,0,0,0,0,1,0.0,1,1,0,30,0,1,0,1,0,25.0,20,90,0,15,1647780,354576,0,0,Bogø Design ApS,8200 Aarhus
160,355409,19776,2023-09-06,2025-04-09,Malles sandwich,0,0,0,36,place_355409@example.com,45355409,DK,DKK,0,0.0,0.0,1970-01-01,0.0,0,0.0,nets,0.0,Shop info filled,0.0,0,0,12191.0,Mr./Mrs. 355409,12192,0,0,Takeaway,Takeaway,1,0,1,0,0,Europe/Copenhagen,da,1,1,0,0,1,1,45355409,place_355409@example.com,https://lovingloyalty.page.link/oLe2tz3feFh5yRYT7,https://lovingloyalty.page.link/V4Xj3gSge5TU472z6,"{""monday"":{""from"":""closed"",""to"":""00.00""},""tues...",0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,1,0,1,40,0,1,60,0,1,1,menu,1,1,1,payment_terminal,0,5.0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,da,0,1,0.0,0,1,1,0,1,0,0,0,0,1,0.0,1,1,0,30,0,1,0,1,0,25.0,20,90,0,15,5973012,35541

In [ ]:
df.to_csv('D:/Deloitte/DIH-X-AUC-Hackathon/data/Inventory Management/dim_places.csv')

In [104]:
data_loader = DataLoader("D:\Deloitte\DIH-X-AUC-Hackathon\data\Inventory Management")
df = data_loader.load_csv("dim_users.csv")


Successfully loaded dim_users.csv: 22955 rows


d:\Deloitte\DIH-X-AUC-Hackathon\src\models\data_loader.py:56: DtypeWarning: Columns (36,40) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, parse_dates=parse_dates)


In [103]:
#knowing the shape, the columns, and the info of the dataset
print(df.shape)
print(df.columns.tolist())
df.head()

(22955, 41)
['id', 'user_id', 'created', 'updated', 'account_closure_requested', 'age_group_id', 'app_version', 'area_id', 'cltv', 'country', 'currency', 'date_of_birth', 'do_not_contact', 'first_name', 'gender_id', 'last_name', 'mobile_phone', 'picture', 'orders', 'redeemed_points', 'referring_user_id', 'referring_place_id', 'savings', 'source', 'type', 'api_key', 'roles', 'email', 'password', 'referral_id', 'otp', 'language', 'email_temp', 'mobile_phone_temp', 'mobile_phone_valid', 'email_valid', 'barcode_scanner_ids', 'receipt_printer_ids', 'payment_terminal_ids', 'notifications', 'favorite_place_ids']


,id,user_id,created,updated,account_closure_requested,age_group_id,app_version,area_id,cltv,country,currency,date_of_birth,do_not_contact,first_name,gender_id,last_name,mobile_phone,picture,orders,redeemed_points,referring_user_id,referring_place_id,savings,source,type,api_key,roles,email,password,referral_id,otp,language,email_temp,mobile_phone_temp,mobile_phone_valid,email_valid,barcode_scanner_ids,receipt_printer_ids,payment_terminal_ids,notifications,favorite_place_ids
0,0,0.0,1571676937,1605685627,0,NaN,NaN,NaN,0.0,DK,DKK,NaN,0,user 0 first_name,NaN,user 0 last_name,450,http://default/sites/default/files/styles/prof...,0,0,NaN,NaN,0.0,NaN,Normal,NaN,anonymous,user_0@example.com,$S$ELsOLSoWL0IdQi.mEK1tXA4Nx0TuPXVTIwwaKeSFL9c...,202257.0,NaN,en,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
1,1,1.0,1571676937,1744002851,0,12033.0,NaN,11470.0,232.0,DK,DKK,387676800.0,0,user 1 first_name,1.0,user 1 last_name,451,http://default/sites/default/files/styles/prof...,2,0,NaN,NaN,0.0,NaN,Normal,bd3f2cec240feda042d65d6d1e981fc6,authenticated|administrator,user_1@example.com,$2y$10$HHa60h/64qTb0KLIu8nUb.8MNlCI4P/vdogslhV...,383219.0,NaN,en,NaN,NaN,0,0,NaN,NaN,NaN,push_orders|push_others|email_orders|email_others,NaN
2,417,417.0,1610542930,1752828166,0,NaN,NaN,NaN,993.0,DK,DKK,NaN,0,user 417 first_name,NaN,user 417 last_name,45417,http://lovingloyalty.com/sites/default/files/s...,22,0,NaN,NaN,2640.0,NaN,Normal,101dfe92603b0cd08448a7e227b4ca28,authenticated|customer|cashier|manager|editor|...,user_417@example.com,$2y$10$Le5rpir1x9LOxcxGQ9JmT.wATxQSS9hozqlGLPg...,254676.0,349460.0,en,NaN,NaN,1,0,NaN,443164|6009243,6009244,push_reviews|push_deals|push_orders|push_other...,NaN
3,423,423.0,1610598865,1744002851,0,NaN,NaN,NaN,0.0,DK,DKK,NaN,0,user 423 first_name,NaN,user 423 last_name,45423,http://default/sites/default/files/styles/prof...,0,0,NaN,NaN,0.0,NaN,Normal,e0677c66e997c605b398f1da5b33307d,authenticated|customer,user_423@example.com,$S$EO07I8QiUMhmZtwcb3OtklQmcACmofRX7cwP13NJd2E...,275349.0,NaN,en,NaN,NaN,0,0,NaN,NaN,NaN,push_reviews|push_deals|push_orders|push_other...,NaN
4,424,424.0,1610598897,1754741871,0,12032.0,1.659650e+12,11457.0,2039.0,DK,DKK,496972800.0,0,user 424 first_name,1.0,user 424 last_name,45424,http://lovingloyalty.com/sites/default/files/s...,75,0,NaN,NaN,9.0,NaN,Normal,8431182a740e189593d8d7159e539c2e,authenticated|customer|cashier|manager|pos_device,user_424@example.com,$2y$10$CPd55Nhvbv.PREE8SP5hpumAgq.K6LfYDKy4K1M...,288986.0,809267.0,en,cashier@lovingloyalty.com,4.522315e+09,0,0,NaN,420542|829147|2388025|2748655|2119442|5935972|...,420539|493523|3291740|5935971|226274|4241778|5...,push_deals|push_orders|push_others|sms_deals|s...,NaN


In [105]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22955 entries, 0 to 22954
Data columns (total 41 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         22955 non-null  int64  
 1   user_id                    22944 non-null  float64
 2   created                    22955 non-null  int64  
 3   updated                    22955 non-null  int64  
 4   account_closure_requested  22955 non-null  int64  
 5   age_group_id               104 non-null    float64
 6   app_version                349 non-null    float64
 7   area_id                    7 non-null      float64
 8   cltv                       22955 non-null  float64
 9   country                    22955 non-null  object 
 10  currency                   22955 non-null  object 
 11  date_of_birth              119 non-null    float64
 12  do_not_contact             22955 non-null  int64  
 13  first_name                 22955 non-null  obj

In [ ]:
#independednt cell to see each columns info (check missing values or 


In [106]:
#dealing with missing data
#convert types
#any missing id will be filled with zero or empty. some columns that their null means no information given like price will be removed
df.drop(['age_group_id','app_version','area_id','date_of_birth','gender_id','referring_user_id','referring_place_id','do_not_contact','mobile_phone','picture','referring_place_id','source','api_key','email','password','referral_id','otp','language','email_temp'],axis=1,inplace=True)

In [108]:
df = df[df.columns.tolist()[0:15]]
df['created'] = pd.to_datetime(df['created'],unit='s').dt.date
df['updated'] = pd.to_datetime(df['updated'],unit='s').dt.date
df.dropna(inplace=True)
cols_to_check = [col for col in df.select_dtypes(include=[np.number]).columns 
                if not any(x in col.lower() for x in ['id', 'created', 'updated'])]
z_scores = np.abs(stats.zscore(df[cols_to_check]))    
df = df[~(z_scores >= 3).any(axis=1)]

In [109]:
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 22762 entries, 0 to 22954
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         22762 non-null  int64  
 1   user_id                    22762 non-null  float64
 2   created                    22762 non-null  object 
 3   updated                    22762 non-null  object 
 4   account_closure_requested  22762 non-null  int64  
 5   cltv                       22762 non-null  float64
 6   country                    22762 non-null  object 
 7   currency                   22762 non-null  object 
 8   first_name                 22762 non-null  object 
 9   last_name                  22762 non-null  object 
 10  orders                     22762 non-null  int64  
 11  redeemed_points            22762 non-null  int64  
 12  savings                    22762 non-null  float64
 13  type                       22762 non-null  object 


In [110]:
df.to_csv('D:/Deloitte/DIH-X-AUC-Hackathon/data/Inventory Management/dim_users.csv')